작업유형 1

In [ ]:
import pandas as pd
a = pd.read_csv('data/mtcars.csv', index_col=0)

# 풀이
from sklearn.preprocessing import MinMaxScaler
scaler = ['qsec']
m = MinMaxScaler()
m.fit(a[scaler])
a['qsec'] = m.transform(a[scaler])
print(len(a[a['qsec'] > 0.5]))



작업유형 2

In [ ]:
# 전처리
# 데이터 파일 읽기 예제
import pandas as pd
# 고객의 상품구매 속성 데이터
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
# 고객 성별 데이터
y_train = pd.read_csv("data/y_train.csv")

# 사용자 코딩
# 1. 결측치
# print(X_train.isna().sum()) # 환불금액 결측치 존재
# print(X_test.isna().sum())
# print(X_train['환불금액'].describe())
# 0으로 채우기
X_train['환불금액'] = X_train['환불금액'].fillna(0)
X_test['환불금액'] = X_test['환불금액'].fillna(0)

# 2. 라벨인코더
from sklearn.preprocessing import LabelEncoder
label = ['주구매상품','주구매지점']
X_train[label] = X_train[label].apply(LabelEncoder().fit_transform)
X_test[label] = X_test[label].apply(LabelEncoder().fit_transform)

# print(X_train['주구매상품'].value_counts()) # 너무 많음
# print(X_train['주구매지점'].value_counts())

# 3. 타입변환,더미
# 주구매지점만 해보기
X_train['주구매지점'] = X_train['주구매지점'].astype('category')
X_test['주구매지점'] = X_test['주구매지점'].astype('category')
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# 4. 파생변수
# 총구매액
X_train['총구매액_qcut'] = pd.qcut(X_train['총구매액'],5,labels=False)
X_test['총구매액_qcut'] = pd.qcut(X_test['총구매액'],5,labels=False)

# 5. 스케일
from sklearn.preprocessing import MinMaxScaler
scaler = ['총구매액','최대구매액','환불금액','내점일수','내점당구매건수','주말방문비율','구매주기']
m = MinMaxScaler()
m.fit(X_train[scaler])
X_train[scaler] = m.transform(X_train[scaler])
X_test[scaler] = m.transform(X_test[scaler])

# 6. 데이터 분리
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_vaild = train_test_split(X_train,y_train['gender'],test_size=0.2,random_state=42,stratify=y_train['gender'])

# print(X_train.shape)
# print(X_valid.shape)

In [ ]:
# 7. 모형 학습, 앙상블
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()
model1.fit(X_train,y_train)
pred1 = pd.DataFrame(model1.predict_proba(X_valid))

from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier()
model2.fit(X_train,y_train)
pred2 = pd.DataFrame(model2.predict_proba(X_valid))

from sklearn.ensemble import VotingClassifier
model3 = VotingClassifier(estimators=[('lr',model1),('rc',model2)],voting='soft')
model3.fit(X_train,y_train)
pred3 = pd.DataFrame(model3.predict_proba(X_valid))

# 8. 평가
from sklearn.metrics import roc_auc_score
# print(roc_auc_score(y_valid,pred1.iloc[:,1]))
# print(roc_auc_score(y_valid,pred2.iloc[:,1]))
# print(roc_auc_score(y_valid,pred3.iloc[:,1]))


# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
result = pd.DataFrame(model3.predict_proba(X_test))
result = result.iloc[:,1]

pd.DataFrame({'cust_id': X_test.cust_id, 'gender': result}).to_csv('003000000.csv', index=False)